In [ ]:
# necessary imports
import numpy as np
import numpy.random as rng 
import tensorflow as tf
import matplotlib.pyplot as plt 
import sklearn.metrics
import keras.layers as layers
import keras.models
import scipy.stats
import keras.utils
import json
import pandas as pd

from graphviz import Source
from string import Template
from PIL import Image

In [ ]:
with open('cars_data.json', 'r') as f:
    cars = json.load(f)
df = pd.DataFrame(cars)

KM_PER_LITER_PER_MILES_PER_GALLON = 0.425144

df['kpl'] = df['Miles_per_Gallon'] * KM_PER_LITER_PER_MILES_PER_GALLON
df['weight'] = df['Weight_in_lbs'] * 0.453592
df = df[~np.isnan(df['kpl']) & ~np.isnan(df['Horsepower'])]

In [ ]:
X = np.array(df['Horsepower'])[:,np.newaxis]
y = np.array(df['kpl'])[:,np.newaxis]

N_HIDDEN = 3

input_node = layers.Input(shape=(X.shape[1],))
hidden_layer = layers.Dense(N_HIDDEN, activation='tanh')(input_node)
output_node = layers.Dense(1, activation='linear')(hidden_layer)
model = keras.models.Model(inputs=input_node, outputs=output_node)
model.compile('sgd', 'mean_squared_error')


In [ ]:
# train the model
Xnormed = (X - np.mean(X)) / np.std(X)
ynormed = np.log(y)

weights = [
    model.get_weights()
]
EPOCHS_PER_REP = 5
REPS = 50

    
hps = np.linspace(0, 250, 1000)[:,np.newaxis]
log_kpls = model.predict((hps - np.mean(X))/np.std(X))

    
for r in range(REPS):
    mse = np.mean(np.square(np.exp(model.predict(Xnormed)) - y))
    print("MSE: %f" % mse)
    
    f, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.plot(df['Horsepower'], df['kpl'], marker='o', linestyle='', markersize=10, color='orange', markeredgecolor='black')
    
    ax.text(0.95, 0.95, 'After %d Epochs\nMSE: %0.2f' % (r*EPOCHS_PER_REP, mse), horizontalalignment='right', color='magenta', verticalalignment='top', fontweight='bold', transform=ax.transAxes, fontsize=28)
    log_kpls = model.predict((hps - np.mean(X))/np.std(X))
    ax.plot(hps, np.exp(log_kpls), linewidth=5, linestyle='--')
    ax.tick_params(axis='x', labelsize=28)
    ax.tick_params(axis='y', labelsize=28)
    ax.set_ylabel('Kilometers per Liter', fontsize=32, fontweight='bold')
    ax.set_xlabel('Horsepower', fontsize=32, fontweight='bold')
    ax.grid('both')
    ax.set_ylim([0, 21])
    plt.savefig('../tmp/nn_fit_%d.png' % (r*EPOCHS_PER_REP), bbox_inches='tight')
    plt.close()
    
    model.fit(x = Xnormed, y=ynormed, verbose=False, epochs=EPOCHS_PER_REP, batch_size=X.shape[0] // 10)
    weights.append(model.get_weights())

    
f, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.plot(df['Horsepower'], df['kpl'], marker='o', linestyle='', markersize=10, color='orange', markeredgecolor='black')

ax.text(0.95, 0.95, 'After %d Epochs\nMSE: %0.2f' % (REPS*EPOCHS_PER_REP, mse), horizontalalignment='right', color='magenta', verticalalignment='top', fontweight='bold', transform=ax.transAxes, fontsize=28)
log_kpls = model.predict((hps - np.mean(X))/np.std(X))
ax.plot(hps, np.exp(log_kpls), linewidth=5, linestyle='--')
ax.tick_params(axis='x', labelsize=28)
ax.tick_params(axis='y', labelsize=28)
ax.set_ylabel('Kilometers per Liter', fontsize=32, fontweight='bold')
ax.set_xlabel('Horsepower', fontsize=32, fontweight='bold')
ax.grid('both')
ax.set_ylim([0, 21])
plt.savefig('../tmp/nn_fit_%d.png' % (REPS*EPOCHS_PER_REP), bbox_inches='tight')
plt.close()

In [ ]:
COLORS = np.array(['red', 'blue', 'blue'])

for i, W in enumerate(weights):

    Wih, bh, Who, bo = W
    
    nWih = np.abs(Wih) * 2
    nbh = np.abs(bh) * 2
    nWho = np.abs(Who) * 2
    nbo = np.abs(bo) * 2
    
    Wih_sign = np.sign(Wih).astype(int) + 1
    bh_sign = np.sign(bh).astype(int) + 1
    Who_sign = np.sign(Who).astype(int) + 1
    bo_sign = np.sign(bo).astype(int) + 1
    
    Wih_color = COLORS[Wih_sign]
    bh_color = COLORS[bh_sign]
    Who_color = COLORS[Who_sign]
    bo_color = COLORS[bo_sign]
    
    print(nWho.shape)
    graph = f"""
    digraph G {{
            graph [dpi=300];
            rankdir=LR
        splines=line
            nodesep=.05;

            node [label="", width=0.5, fixedsize=true, penwidth=2];

            subgraph cluster_0 {{
            color=white;
                    node [style=solid,color=purple, shape=circle];
            x1 x2;
            label = "Input Layer";
        }}

        subgraph cluster_1 {{
            color=white;
            node [style=solid,color=orange, shape=circle];
            a12 a22 a32 a42;
            label = "Hidden Layer";
        }}

        subgraph cluster_2 {{
            color=white;
            node [style=solid,color=seagreen2, shape=circle];
            y;
            label="Output Layer";
        }}

            x1 [label=1]
            x2 [label=HP]
            a12 [label=1]
            a22 [label=<h<sub>1</sub>>]
            a32 [label=<h<sub>2</sub>>]
            a42 [label=<h<sub>3</sub>>]
            y [label=KPL]

            x1 -> a22 [penwidth={nbh[0]}, color={bh_color[0]}];
            x1 -> a32 [penwidth={nbh[1]}, color={bh_color[1]}];
            x1 -> a42 [penwidth={nbh[2]}, color={bh_color[2]}];


            x2 -> a22  [penwidth={nWih[0, 0]}, color={Wih_color[0, 0]}];
            x2 -> a32  [penwidth={nWih[0, 1]}, color={Wih_color[0, 1]}];
            x2 -> a42  [penwidth={nWih[0, 2]}, color={Wih_color[0, 2]}];


            a12 -> y [penwidth={nbo[0]}, color={bo_color[00]}]
            a22 -> y [penwidth={nWho[0, 0]}, color={Who_color[0, 0]}]
            a32 -> y [penwidth={nWho[1, 0]}, color={Who_color[1, 0]}]
            a42 -> y [penwidth={nWho[2, 0]}, color={Who_color[2, 0]}]



    }} """ 
    
    dot = Source(graph)
    dot.render(filename='../tmp/intermediate_net_%d' % (i*EPOCHS_PER_REP), format='png')

In [ ]:
for r in range(len(weights)):
    epochs = r*EPOCHS_PER_REP
    im1 = Image.open('../tmp/nn_fit_%d.png' % epochs)
    im2 = Image.open('../tmp/intermediate_net_%d.png' % epochs)
    im2.thumbnail(im1.size, Image.ANTIALIAS)

    dst = Image.new('RGB', (im1.width + im2.width, im1.height), "white")
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, (im1.height - im2.height)//2))
    dst.save('../tmp/fit_progress_%d.png' % (epochs))